# UFoE Phase 5b-2: ColabFold Multimer 검증
## 곤 코어 / 건 표면 / 균형 설계 — 최종 검증

### 확인 사항 3가지:
1. **Inter-chain salt bridge** (e↔g') 정렬 여부
2. **a,d registry alignment** 보존 여부  
3. **Oligomeric state** 유지 여부 (C 그룹이 dimer 유지하는지)

### 실험 그룹:
- **A (WT)**: RMKQLEDKVEELLSKNYHLENEVARLKKLVGER
- **B (곤강화)**: LMKQLEDIVEELLSLNYHLEINEVALKKLLGEL
- **C (건강화)**: REKSLEDKVEELLSKEYKLENEVERKKLEGEK

---

## Step 1: ColabFold 설치 및 설정

In [ ]:
# ColabFold 설치 (Google Colab에서 실행)
!pip install -q colabfold[alphafold]@git+https://github.com/sokrypton/ColabFold
!pip install -q biopython matplotlib numpy

## Step 2: 확정 서열 정의

In [ ]:
import os
import json
import numpy as np

# === 확정 서열 ===
SEQUENCES = {
    'A_WT': 'RMKQLEDKVEELLSKNYHLENEVARLKKLVGER',
    'B_GON': 'LMKQLEDIVEELLSLNYHLEINEVALKKLLGEL',
    'C_GEON': 'REKSLEDKVEELLSKEYKLENEVERKKLEGEK'
}

# === 곤감리건 매핑 ===
GGRL_MAP = {
    'A':'gon','L':'gon','V':'gon','I':'gon','F':'gon','M':'gon','W':'gon',
    'G':'gam','C':'gam','P':'gam',
    'S':'ri','T':'ri','N':'ri','Q':'ri','H':'ri','Y':'ri',
    'E':'geon','D':'geon','K':'geon','R':'geon'
}

# === Heptad Registry ===
def get_heptad_registry(seq_length, start_position='d'):
    heptad = ['d', 'e', 'f', 'g', 'a', 'b', 'c']
    start_offset = heptad.index(start_position)
    return {i: heptad[(i + start_offset) % 7] for i in range(seq_length)}

def get_positions_by_role(registry):
    roles = {'a': [], 'd': [], 'e': [], 'g': [], 'bcf': []}
    for idx, letter in registry.items():
        if letter in ('a', 'd', 'e', 'g'):
            roles[letter].append(idx)
        else:
            roles['bcf'].append(idx)
    return roles

# 서열 검증
for name, seq in SEQUENCES.items():
    reg = get_heptad_registry(len(seq))
    pos = get_positions_by_role(reg)
    ggrl = [GGRL_MAP[aa] for aa in seq]
    n = len(ggrl)
    ratio = {t: round(ggrl.count(t)/n*100, 1) for t in ['gon','gam','ri','geon']}
    print(f'{name} ({len(seq)}aa): gon={ratio["gon"]}% gam={ratio["gam"]}% ri={ratio["ri"]}% geon={ratio["geon"]}%')
    print(f'  a positions: {[seq[i] for i in pos["a"]]}')
    print(f'  d positions: {[seq[i] for i in pos["d"]]}')
    print(f'  e positions: {[seq[i] for i in pos["e"]]}')
    print(f'  g positions: {[seq[i] for i in pos["g"]]}')
    print()

## Step 3: FASTA 파일 생성 및 ColabFold 실행

In [ ]:
# FASTA 파일 생성 (ColabFold Multimer용: ':' 로 체인 구분)
os.makedirs('gcn4_colabfold', exist_ok=True)

for name, seq in SEQUENCES.items():
    fasta_path = f'gcn4_colabfold/{name}.fasta'
    # Homodimer: 동일 서열 2개
    with open(fasta_path, 'w') as f:
        f.write(f'>{name}\n')
        f.write(f'{seq}:{seq}\n')  # ':' = chain separator
    print(f'Created: {fasta_path}')
    print(f'  Content: {seq}:{seq}')
    print()

In [ ]:
# === ColabFold Multimer 실행 ===
# 각 그룹별로 실행 (시간: 그룹당 ~5-10분)

for name in SEQUENCES.keys():
    fasta = f'gcn4_colabfold/{name}.fasta'
    outdir = f'gcn4_colabfold/output_{name}'
    os.makedirs(outdir, exist_ok=True)
    
    print(f'\n=== Running ColabFold for {name} ===')
    !colabfold_batch {fasta} {outdir} \
        --num-models 5 \
        --num-recycles 3 \
        --model-type alphafold2_multimer_v3 \
        --templates \
        --amber
    
    print(f'Done: {name}')

## Step 4: PDB 파일 로드

ColabFold가 완료되면 rank_001 PDB를 사용합니다.

In [ ]:
import glob
from Bio.PDB import PDBParser

def find_best_pdb(output_dir):
    """rank_001 또는 최고 pLDDT PDB 찾기"""
    pdbs = glob.glob(os.path.join(output_dir, '*rank_001*.pdb'))
    if not pdbs:
        pdbs = glob.glob(os.path.join(output_dir, '*.pdb'))
    if not pdbs:
        # relaxed 버전 확인
        pdbs = glob.glob(os.path.join(output_dir, '*relaxed*.pdb'))
    return pdbs[0] if pdbs else None

pdb_files = {}
for name in SEQUENCES.keys():
    outdir = f'gcn4_colabfold/output_{name}'
    pdb = find_best_pdb(outdir)
    if pdb:
        pdb_files[name] = pdb
        print(f'{name}: {pdb}')
    else:
        print(f'{name}: PDB not found! Check output directory.')

# 체인 수 확인
parser = PDBParser(QUIET=True)
for name, pdb_path in pdb_files.items():
    structure = parser.get_structure(name, pdb_path)
    chains = list(structure[0].get_chains())
    print(f'{name}: {len(chains)} chains ({[c.id for c in chains]})')

## Step 5: 분석 함수 정의

In [ ]:
# ============================================================
# 분석 1: Inter-chain Salt Bridge (e↔g' registry 필터)
# ============================================================

def detect_salt_bridges(pdb_file, sequence, cutoff=4.0):
    """
    Inter-chain salt bridge 검출.
    정규(e↔g') vs 비정규 분리.
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('dimer', pdb_file)
    chains = list(structure[0].get_chains())
    
    if len(chains) < 2:
        print('ERROR: Need 2 chains for dimer analysis')
        return None
    
    registry = get_heptad_registry(len(sequence))
    positions = get_positions_by_role(registry)
    e_set = set(positions['e'])
    g_set = set(positions['g'])
    
    neg_atoms = {'GLU': ['OE1', 'OE2'], 'ASP': ['OD1', 'OD2']}
    pos_atoms = {'LYS': ['NZ'], 'ARG': ['NH1', 'NH2']}
    
    regular = []
    irregular = []
    
    res_a = list(chains[0].get_residues())
    res_b = list(chains[1].get_residues())
    
    for i, ra in enumerate(res_a):
        rn_a = ra.get_resname()
        is_neg_a = rn_a in neg_atoms
        is_pos_a = rn_a in pos_atoms
        if not (is_neg_a or is_pos_a):
            continue
        
        for j, rb in enumerate(res_b):
            rn_b = rb.get_resname()
            is_neg_b = rn_b in neg_atoms
            is_pos_b = rn_b in pos_atoms
            if not (is_neg_b or is_pos_b):
                continue
            
            # 반대 전하만
            if not ((is_neg_a and is_pos_b) or (is_pos_a and is_neg_b)):
                continue
            
            # 원자 수준 최소 거리
            a_names = neg_atoms.get(rn_a, []) + pos_atoms.get(rn_a, [])
            b_names = neg_atoms.get(rn_b, []) + pos_atoms.get(rn_b, [])
            
            min_dist = 999
            for an in a_names:
                for bn in b_names:
                    if an in ra and bn in rb:
                        d = ra[an] - rb[bn]
                        if d < min_dist:
                            min_dist = d
            
            if min_dist < cutoff:
                info = {
                    'chain_a_res': f'{rn_a}{i+1}',
                    'chain_b_res': f'{rn_b}{j+1}',
                    'distance': round(min_dist, 2),
                    'heptad_a': registry.get(i, '?'),
                    'heptad_b': registry.get(j, '?')
                }
                
                is_eg = (i in e_set and j in g_set)
                is_ge = (i in g_set and j in e_set)
                
                if is_eg or is_ge:
                    info['type'] = 'REGULAR (e↔g\')'
                    regular.append(info)
                else:
                    info['type'] = 'irregular'
                    irregular.append(info)
    
    return {
        'regular': regular,
        'irregular': irregular,
        'regular_count': len(regular),
        'irregular_count': len(irregular),
        'total': len(regular) + len(irregular)
    }

In [ ]:
# ============================================================
# 분석 2: a,d Core Packing (Cβ-Cβ distance)
# ============================================================

def analyze_core_packing(pdb_file, sequence):
    """
    Chain A의 a,d ↔ Chain B의 a,d 위치 Cβ-Cβ 거리.
    
    정상 범위:
    - d↔d': 5-8Å (knobs-into-holes)
    - a↔a': 6-8Å
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('dimer', pdb_file)
    chains = list(structure[0].get_chains())
    
    if len(chains) < 2:
        return None
    
    registry = get_heptad_registry(len(sequence))
    positions = get_positions_by_role(registry)
    
    def get_cb(residue):
        if 'CB' in residue:
            return residue['CB'].get_vector().get_array()
        elif 'CA' in residue:
            return residue['CA'].get_vector().get_array()
        return None
    
    res_a = list(chains[0].get_residues())
    res_b = list(chains[1].get_residues())
    
    distances = {'a_a': [], 'd_d': [], 'a_d': []}
    details = {'a_a': [], 'd_d': [], 'a_d': []}
    
    # d↔d' (같은 heptad 위치끼리 — 가장 중요)
    for pos in positions['d']:
        if pos < len(res_a) and pos < len(res_b):
            cb_a = get_cb(res_a[pos])
            cb_b = get_cb(res_b[pos])
            if cb_a is not None and cb_b is not None:
                dist = np.linalg.norm(cb_a - cb_b)
                distances['d_d'].append(round(dist, 2))
                details['d_d'].append({
                    'pos': pos+1,
                    'aa_a': sequence[pos],
                    'aa_b': sequence[pos],
                    'dist': round(dist, 2)
                })
    
    # a↔a'
    for pos in positions['a']:
        if pos < len(res_a) and pos < len(res_b):
            cb_a = get_cb(res_a[pos])
            cb_b = get_cb(res_b[pos])
            if cb_a is not None and cb_b is not None:
                dist = np.linalg.norm(cb_a - cb_b)
                distances['a_a'].append(round(dist, 2))
                details['a_a'].append({
                    'pos': pos+1,
                    'aa_a': sequence[pos],
                    'aa_b': sequence[pos],
                    'dist': round(dist, 2)
                })
    
    # a↔d' (knobs-into-holes)
    for pos_a in positions['a']:
        for pos_d in positions['d']:
            if pos_a < len(res_a) and pos_d < len(res_b):
                cb_a = get_cb(res_a[pos_a])
                cb_b = get_cb(res_b[pos_d])
                if cb_a is not None and cb_b is not None:
                    dist = np.linalg.norm(cb_a - cb_b)
                    if dist < 12.0:
                        distances['a_d'].append(round(dist, 2))
    
    result = {}
    for key in distances:
        if distances[key]:
            result[key] = {
                'mean': round(np.mean(distances[key]), 2),
                'std': round(np.std(distances[key]), 2),
                'min': round(min(distances[key]), 2),
                'max': round(max(distances[key]), 2),
                'values': distances[key],
                'details': details.get(key, [])
            }
    
    return result

In [ ]:
# ============================================================
# 분석 3: Crossing Angle (DSSP helix-only PCA)
# ============================================================

def calculate_crossing_angle(pdb_file):
    """
    DSSP helix 잔기만으로 PCA → helix 축 → 교차각.
    DSSP 실패 시 전체 CA fallback.
    
    코일드-코일 정상 범위: 18-25°
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('dimer', pdb_file)
    model = structure[0]
    chains = list(model.get_chains())
    
    if len(chains) < 2:
        return None
    
    # DSSP 시도
    helix_res = {0: set(), 1: set()}
    dssp_used = False
    try:
        from Bio.PDB.DSSP import DSSP
        dssp = DSSP(model, pdb_file, dssp='mkdssp')
        for key, value in dssp:
            chain_id = key[0]
            res_id = key[1][1]
            ss = value[2]
            if ss == 'H':
                ci = 0 if chain_id == chains[0].id else 1
                helix_res[ci].add(res_id)
        dssp_used = True
    except:
        for ci, chain in enumerate(chains):
            for res in chain:
                helix_res[ci].add(res.get_id()[1])
    
    def get_axis(chain, h_set):
        ca = []
        for res in chain:
            if res.get_id()[1] in h_set and 'CA' in res:
                ca.append(res['CA'].get_vector().get_array())
        if len(ca) < 5:
            return None, None
        ca = np.array(ca)
        center = ca.mean(axis=0)
        centered = ca - center
        cov = np.cov(centered.T)
        eigvals, eigvecs = np.linalg.eigh(cov)
        return eigvecs[:, np.argmax(eigvals)], center
    
    ax_a, c_a = get_axis(chains[0], helix_res[0])
    ax_b, c_b = get_axis(chains[1], helix_res[1])
    
    if ax_a is None or ax_b is None:
        return None
    
    dot = np.dot(ax_a, ax_b)
    angle = np.degrees(np.arccos(min(1.0, abs(dot))))
    
    return {
        'crossing_angle': round(angle, 1),
        'orientation': 'parallel' if dot > 0 else 'antiparallel',
        'helix_A': len(helix_res[0]),
        'helix_B': len(helix_res[1]),
        'dssp_used': dssp_used
    }

In [ ]:
# ============================================================
# 분석 4: Oligomeric State 확인
# ============================================================

def check_oligomeric_state(pdb_file):
    """
    두 체인이 실제로 dimer를 형성하는지 확인.
    - 체인 간 최소 거리
    - 접촉 잔기 수
    - 인터페이스 면적 추정
    
    C 그룹 핵심: dimer 유지 vs monomer-like collapse
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('dimer', pdb_file)
    chains = list(structure[0].get_chains())
    
    if len(chains) < 2:
        return {'state': 'MONOMER', 'chains': 1}
    
    # 체인 간 CA-CA 거리
    ca_a = [r['CA'].get_vector().get_array() for r in chains[0] if 'CA' in r]
    ca_b = [r['CA'].get_vector().get_array() for r in chains[1] if 'CA' in r]
    
    ca_a = np.array(ca_a)
    ca_b = np.array(ca_b)
    
    # 모든 쌍 거리
    from scipy.spatial.distance import cdist
    dist_matrix = cdist(ca_a, ca_b)
    
    min_dist = dist_matrix.min()
    contact_count = (dist_matrix < 8.0).sum()  # 8Å 이내 접촉
    close_count = (dist_matrix < 5.0).sum()    # 5Å 이내 밀접
    
    # 체인 중심 간 거리
    center_a = ca_a.mean(axis=0)
    center_b = ca_b.mean(axis=0)
    center_dist = np.linalg.norm(center_a - center_b)
    
    # 판정
    if min_dist > 12.0:
        state = 'SEPARATED (no contact)'
    elif contact_count < 5:
        state = 'WEAK DIMER'
    elif close_count > 10:
        state = 'TIGHT DIMER'
    else:
        state = 'DIMER'
    
    return {
        'state': state,
        'chains': len(chains),
        'min_distance': round(min_dist, 2),
        'contacts_8A': int(contact_count),
        'contacts_5A': int(close_count),
        'center_distance': round(center_dist, 2)
    }

In [ ]:
# ============================================================
# 분석 5: SASA (곤 매몰 최종 확인)
# ============================================================

def analyze_sasa(pdb_file, sequence):
    """
    Dimer에서 곤감리건 타입별 상대 SASA.
    Phase 5a에서 불가능했던 곤 매몰 검증.
    """
    from Bio.PDB.SASA import ShrakeRupley
    
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('dimer', pdb_file)
    sr = ShrakeRupley()
    sr.compute(structure[0], level='R')
    
    max_sasa = {
        'A':129,'R':274,'N':195,'D':193,'C':167,
        'E':223,'Q':225,'G':104,'H':224,'I':197,
        'L':201,'K':236,'M':224,'F':240,'P':159,
        'S':155,'T':172,'W':285,'Y':263,'V':174
    }
    
    sasa_by_type = {'gon':[], 'gam':[], 'ri':[], 'geon':[]}
    seq_combined = sequence + sequence  # dimer
    
    for i, res in enumerate(structure[0].get_residues()):
        if i < len(seq_combined):
            aa = seq_combined[i]
            ggrl = GGRL_MAP.get(aa)
            if ggrl:
                rel = res.sasa / max_sasa.get(aa, 200)
                sasa_by_type[ggrl].append(rel)
    
    result = {}
    for t in ['gon', 'gam', 'ri', 'geon']:
        if sasa_by_type[t]:
            result[t] = {
                'mean': round(np.mean(sasa_by_type[t]), 3),
                'std': round(np.std(sasa_by_type[t]), 3),
                'n': len(sasa_by_type[t])
            }
    
    # 나이테 순서 판정: 곤 < 리 < 건
    if 'gon' in result and 'geon' in result:
        result['naitae_order'] = result['gon']['mean'] < result.get('ri', {}).get('mean', 999) < result['geon']['mean']
    
    return result

## Step 6: 종합 분석 실행

In [ ]:
# ============================================================
# 종합 실행
# ============================================================

all_results = {}

for name in SEQUENCES.keys():
    if name not in pdb_files:
        print(f'\n⚠️ {name}: PDB not found, skipping')
        continue
    
    pdb = pdb_files[name]
    seq = SEQUENCES[name]
    
    print(f'\n{"="*60}')
    print(f'=== {name} ===')
    print(f'{"="*60}')
    
    # 1. Oligomeric State
    oligo = check_oligomeric_state(pdb)
    print(f'\n[Oligomeric State]')
    print(f'  State: {oligo["state"]}')
    print(f'  Min distance: {oligo["min_distance"]}Å')
    print(f'  Contacts (8Å): {oligo["contacts_8A"]}')
    print(f'  Center distance: {oligo["center_distance"]}Å')
    
    # 2. Salt Bridge
    sb = detect_salt_bridges(pdb, seq)
    print(f'\n[Salt Bridges]')
    if sb:
        print(f'  Regular (e↔g\'): {sb["regular_count"]}')
        for b in sb['regular']:
            print(f'    {b["chain_a_res"]}({b["heptad_a"]}) ↔ {b["chain_b_res"]}({b["heptad_b"]}) = {b["distance"]}Å')
        print(f'  Irregular: {sb["irregular_count"]}')
        print(f'  Total: {sb["total"]}')
    
    # 3. Core Packing
    cp = analyze_core_packing(pdb, seq)
    print(f'\n[Core Packing (Cβ-Cβ)]')
    if cp:
        for key in ['d_d', 'a_a', 'a_d']:
            if key in cp:
                print(f'  {key}: mean={cp[key]["mean"]}Å std={cp[key]["std"]}Å min={cp[key]["min"]}Å max={cp[key]["max"]}Å')
                if 'details' in cp[key]:
                    for d in cp[key]['details']:
                        print(f'    pos{d["pos"]}: {d["aa_a"]}↔{d["aa_b"]} = {d["dist"]}Å')
    
    # 4. Crossing Angle
    ca = calculate_crossing_angle(pdb)
    print(f'\n[Crossing Angle]')
    if ca:
        print(f'  Angle: {ca["crossing_angle"]}°')
        print(f'  Orientation: {ca["orientation"]}')
        print(f'  DSSP used: {ca["dssp_used"]}')
    
    # 5. SASA
    sasa = analyze_sasa(pdb, seq)
    print(f'\n[SASA by GGRL type]')
    if sasa:
        for t in ['gon', 'gam', 'ri', 'geon']:
            if t in sasa:
                print(f'  {t}: mean={sasa[t]["mean"]} (n={sasa[t]["n"]})')
        if 'naitae_order' in sasa:
            print(f'  나이테 순서 (곤<리<건): {"✅" if sasa["naitae_order"] else "❌"}')
    
    all_results[name] = {
        'oligomeric': oligo,
        'salt_bridges': sb,
        'core_packing': cp,
        'crossing_angle': ca,
        'sasa': sasa
    }

## Step 7: 판정

In [ ]:
# ============================================================
# 최종 판정
# ============================================================

print('\n' + '='*60)
print('=== Phase 5b-2 ColabFold Multimer — 최종 판정 ===')
print('='*60)

r = all_results

# === 판정 1: 곤 코어 원리 ===
print('\n[1] 곤 코어 원리')
try:
    b_dd = r['B_GON']['core_packing']['d_d']['mean']
    a_dd = r['A_WT']['core_packing']['d_d']['mean']
    c_dd = r['C_GEON']['core_packing']['d_d']['mean']
    gon_pass = b_dd < a_dd < c_dd
    print(f'  d↔d\' mean: B={b_dd}Å < A={a_dd}Å < C={c_dd}Å')
    print(f'  → {"PASS ✅" if gon_pass else "FAIL ❌"}')
except:
    print('  → 데이터 부족')
    gon_pass = False

# === 판정 2: 건 표면 원리 ===
print('\n[2] 건 표면 원리')
try:
    a_sb = r['A_WT']['salt_bridges']['regular_count']
    b_sb = r['B_GON']['salt_bridges']['regular_count']
    c_sb = r['C_GEON']['salt_bridges']['regular_count']
    c_sb_irreg = r['C_GEON']['salt_bridges']['irregular_count']
    # 수정된 기준: A ≥ C ≥ B 또는 C > A ≥ B
    geon_pass = (a_sb >= b_sb) and (c_sb + c_sb_irreg >= b_sb)
    print(f'  Regular salt bridges: A={a_sb}, B={b_sb}, C={c_sb}')
    print(f'  C irregular: {c_sb_irreg}')
    print(f'  → {"PASS ✅" if geon_pass else "FAIL ❌"}')
except:
    print('  → 데이터 부족')
    geon_pass = False

# === 판정 3: 균형 설계 조건 ===
print('\n[3] 균형 설계 조건')
try:
    a_angle = r['A_WT']['crossing_angle']['crossing_angle']
    balance_pass = 15 <= a_angle <= 28
    print(f'  WT crossing angle: {a_angle}° (기준: 18-25°)')
    print(f'  → {"PASS ✅" if balance_pass else "FAIL ❌"}')
except:
    print('  → 데이터 부족')
    balance_pass = False

# === 판정 4: C 그룹 oligomeric state ===
print('\n[4] C 그룹 dimer 유지 여부')
try:
    c_state = r['C_GEON']['oligomeric']['state']
    c_contacts = r['C_GEON']['oligomeric']['contacts_8A']
    print(f'  State: {c_state}')
    print(f'  Contacts (8Å): {c_contacts}')
    if 'SEPARATED' in c_state:
        print(f'  → C는 dimer 유지 실패 (monomer-like collapse)')
    elif 'WEAK' in c_state:
        print(f'  → C는 약한 dimer (건 과다 → 불안정)')
    else:
        print(f'  → C는 dimer 유지')
except:
    print('  → 데이터 부족')

# === 판정 5: SASA 나이테 ===
print('\n[5] SASA 나이테 (곤 < 리 < 건)')
try:
    a_sasa = r['A_WT']['sasa']
    naitae = a_sasa.get('naitae_order', False)
    print(f'  WT: 곤={a_sasa["gon"]["mean"]}, 리={a_sasa.get("ri",{}).get("mean","N/A")}, 건={a_sasa["geon"]["mean"]}')
    print(f'  → {"PASS ✅" if naitae else "FAIL ❌"}')
except:
    print('  → 데이터 부족')

# === 종합 ===
print('\n' + '='*60)
total_pass = sum([gon_pass, geon_pass, balance_pass])
print(f'종합: {total_pass}/3 핵심 판정 PASS')
if total_pass == 3:
    print('\n🎉 Phase 5b-2 완료!')
    print('→ 곤 코어 / 건 표면 / 균형 설계 3원리 고정')
    print('→ Phase 5c (β-sheet dimer) 진행 가능')
else:
    print(f'\n⚠️ {3-total_pass}개 항목 추가 검증 필요')
print('='*60)

In [ ]:
# 결과 저장
with open('gcn4_colabfold/phase5b2_colabfold_results.json', 'w') as f:
    json.dump(all_results, f, indent=2, default=str)

print('Results saved to gcn4_colabfold/phase5b2_colabfold_results.json')

## Step 8: 시각화

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

groups = ['A_WT', 'B_GON', 'C_GEON']
labels = ['A (WT)', 'B (곤↑)', 'C (건↑)']
colors = ['#3498db', '#e74c3c', '#2ecc71']

# Plot 1: d↔d' Cβ distances
ax = axes[0]
dd_means = []
dd_stds = []
for g in groups:
    if g in all_results and all_results[g]['core_packing'] and 'd_d' in all_results[g]['core_packing']:
        dd_means.append(all_results[g]['core_packing']['d_d']['mean'])
        dd_stds.append(all_results[g]['core_packing']['d_d']['std'])
    else:
        dd_means.append(0)
        dd_stds.append(0)

bars = ax.bar(labels, dd_means, yerr=dd_stds, color=colors, capsize=5, edgecolor='black', linewidth=0.5)
ax.axhspan(5, 8, alpha=0.1, color='green', label='Normal range')
ax.set_ylabel('d↔d\' Cβ-Cβ Distance (Å)')
ax.set_title('Core Packing (곤 코어)')
ax.legend()

# Plot 2: Salt bridges
ax = axes[1]
sb_regular = []
sb_irregular = []
for g in groups:
    if g in all_results and all_results[g]['salt_bridges']:
        sb_regular.append(all_results[g]['salt_bridges']['regular_count'])
        sb_irregular.append(all_results[g]['salt_bridges']['irregular_count'])
    else:
        sb_regular.append(0)
        sb_irregular.append(0)

x = np.arange(len(labels))
ax.bar(x, sb_regular, 0.4, label='Regular (e↔g\')', color=colors, edgecolor='black', linewidth=0.5)
ax.bar(x, sb_irregular, 0.4, bottom=sb_regular, label='Irregular', color=[c+'80' for c in colors], edgecolor='black', linewidth=0.5, alpha=0.5)
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_ylabel('Salt Bridge Count')
ax.set_title('Salt Bridges (건 표면)')
ax.legend()

# Plot 3: Crossing angle
ax = axes[2]
angles = []
for g in groups:
    if g in all_results and all_results[g]['crossing_angle']:
        angles.append(all_results[g]['crossing_angle']['crossing_angle'])
    else:
        angles.append(0)

ax.bar(labels, angles, color=colors, edgecolor='black', linewidth=0.5)
ax.axhspan(18, 25, alpha=0.15, color='green', label='Normal range (18-25°)')
ax.set_ylabel('Crossing Angle (°)')
ax.set_title('Helix Crossing (균형)')
ax.legend()

plt.tight_layout()
plt.savefig('gcn4_colabfold/phase5b2_summary.png', dpi=150, bbox_inches='tight')
plt.show()
print('Figure saved.')